In [ ]:
import tensorflow.keras.layers as tfl
from tensorflow.keras import Model, Input, utils, regularizers
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow import reshape
from keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.data import Dataset
import h5py
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def backbone_model(
    image_shape = (224, 224 , 3),
    category_depth = 9,
    trainable = False):

    '''
    image_shape - the shape of the input images
    category_depth - the position of the layer where we using as the input of category
    trainable - whether to train the backbone or not
    ************************************
    return - A model that outputs the encoding for category classification task and grading task
    '''

    backbone = MobileNetV3Small(
    input_shape= image_shape,
    include_top= False,
    weights= 'imagenet',
    input_tensor= None,
    dropout_rate= 0.2,
    include_preprocessing= True)

    backbone.trainable = trainable

    last_layer = backbone.get_layer(backbone.layers[-1].name).output

    conv_output = backbone.get_layer(f'expanded_conv_{str(category_depth)}/Add').output

    return Model(inputs = backbone.input, outputs = [last_layer, conv_output])

In [ ]:
def flat_model(
    image_shape = (224, 224 , 3),
    num_class = 4,
    L2 = 0
):

    '''
    image_shape - the shape of the input images
    num_class - the number ouput nodes or number of classes
    L2 - the weight decay coefficient
    ************************************
    return - A model that outputs the predicted probabilities for each class
    '''
    #initialize the backbone model
    backbone = backbone_model(
    image_shape = image_shape,
    trainable = False)

    backbone.trainable = False

    input = Input(shape = image_shape)

    [last_layer, _] = backbone(input, training = False)


    X = tfl.Flatten()(last_layer)
    X = tfl.Dense(200, activation = 'relu', kernel_regularizer= regularizers.l2(L2))(X)
    X = tfl.Dropout(0.2)(X)
    X = tfl.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(L2))(X)
    X = tfl.Dropout(0.2)(X)
    outputs = tfl.Dense(num_class, activation = 'softmax', name = 'category')(X)

    model = Model(inputs = input, outputs = outputs)

    return model

In [ ]:
#customize accuracy function to validate the ability of predict category and quality independently

def category_accuracy(y_true, y_pred):
    y_true = tf.argmax(y_true, axis = 1)//3
    y_pred = tf.argmax(y_pred, axis = 1)//3
    return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred), tf.float32))

def grade_accuracy(y_true, y_pred):
    y_true = tf.argmax(y_true, axis = 1).numpy()%3
    y_pred = tf.argmax(y_pred, axis = 1).numpy()%3
    return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred), tf.float32))

In [ ]:
file_path = '/content/drive/MyDrive/DAP391m_G06_AI1807/Data/TRAIN DATA/flat_15_224x224_21952.h5'
# file_path = '/content/drive/MyDrive/DAP391m_G06_AI1807/Data/TRAIN DATA/hierachical_6_3_224x224_30350.h5'

with h5py.File(file_path, 'r') as hf:
    X = hf['images'][()]
    Class = hf['class'][()]
print(X.shape)
X_shape = X.shape
print(Class.shape)
num_class = len(set(Class))
print(num_class)
Labels = np.array(reshape(utils.to_categorical(Class,num_class),(-1,num_class)))

print(Labels.shape)

(21952, 224, 224, 3)
(21952,)
12
(21952, 12)


In [ ]:
X_train, X_test, Labels_train, Labels_test = train_test_split(
    X, Labels, test_size=0.2, random_state=42
)
del Class
del X
train_set = Dataset.from_tensor_slices((X_train, Labels_train))
train_set = train_set.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
train_set = train_set.batch(32)

test_set = Dataset.from_tensor_slices((X_test, Labels_test))
test_set = test_set.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
test_set = test_set.batch(32)

del X_train
del Labels_train
del Labels_test

In [ ]:
lr_schedule = ExponentialDecay(
    2e-4,
    decay_steps=2500,
    decay_rate=0.7,
    staircase=True
)
train_set = train_set.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

cardinary_model = flat_model(image_shape=(X_shape[1],X_shape[2],3), num_class=num_class, L2=0.2)
cardinary_model.compile(optimizer = Adam(learning_rate=lr_schedule,), loss = 'categorical_crossentropy', metrics = ['accuracy'])
cardinary_model.fit(train_set, epochs = 25)

4334752/4334752 [==============================] - 1s 0us/step
Epoch 1/25
549/549 [==============================] - 21s 22ms/step - loss: 23.1192 - accuracy: 0.7165
Epoch 2/25
549/549 [==============================] - 11s 20ms/step - loss: 2.6322 - accuracy: 0.8071
Epoch 3/25
549/549 [==============================] - 12s 22ms/step - loss: 1.5830 - accuracy: 0.8066
Epoch 4/25
549/549 [==============================] - 12s 22ms/step - loss: 1.5461 - accuracy: 0.7988
Epoch 5/25
549/549 [==============================] - 11s 21ms/step - loss: 1.4550 - accuracy: 0.8077
Epoch 6/25
549/549 [==============================] - 11s 20ms/step - loss: 1.3180 - accuracy: 0.8250
Epoch 7/25
549/549 [==============================] - 11s 21ms/step - loss: 1.3203 - accuracy: 0.8295
Epoch 8/25
549/549 [==============================] - 11s 20ms/step - loss: 1.3207 - accuracy: 0.8258
Epoch 9/25
549/549 [==============================] - 11s 20ms/step - loss: 1.2929 - accuracy: 0.8328
Epoch 10/25
549/54

In [ ]:
all_X = []
all_y = []
for batch in test_set:
    X, y = batch  # Unpack the batch into X (inputs) and y (labels)
    # Collect batch of inputs
    all_X.append(X.numpy())  # Assuming X is a TensorFlow tensor, convert to NumPy if needed
    all_y.append(y.numpy())  # Assuming y is a TensorFlow tensor, convert to NumPy if needed
# Concatenate all batches into single arrays or lists
all_X = tf.concat(all_X, axis=0)  # Concatenate along batch dimension if X is a tensor
all_y = tf.concat(all_y, axis=0)  # Concatenate along batch dimension if y is a tensor

In [ ]:
results = cardinary_model.evaluate(test_set)
y_pred = cardinary_model.predict(all_X)
print('Category_Accuracy :', category_accuracy(all_y, y_pred).numpy())
print('Grade_Accuracy :', grade_accuracy(all_y, y_pred).numpy())

138/138 [==============================] - 3s 15ms/step
Category_Accuracy : 0.9553632
Grade_Accuracy : 0.93236166
